# Import

In [7]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from tqdm import tqdm

In [8]:
tqdm.pandas()

# Read data

In [9]:
ag_news_df = pd.read_csv("../../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [10]:
with open("../../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [11]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [12]:
model_nums = 2
vector_model_num = 0

In [13]:
def getKMeans(vectors, seed, path):
    model = KMeans(
        n_clusters=4, random_state=seed
    )
    model.fit(vectors)
    pickle.dump(model, open(path, "wb"))
    pred = model.predict(vectors)
    return pred

In [14]:
vectors_path = "../../data/AgNews/doc2vec/vector/"
models_path = "../../data/AgNews/doc2vec/KMeans/model/"
pred_path = "../../data/AgNews/doc2vec/KMeans/pred/"
for vector_size in tqdm(vector_sizes):
    for model_num in range(model_nums):
        vector_path = f"{vectors_path}{vector_size}/normalized/{vector_model_num}.csv"
        vectors = np.loadtxt(vector_path, delimiter=",")
        
        os.makedirs(f"{models_path}{vector_size}", exist_ok=True)
        pred = getKMeans(vectors, seed=model_num, path=f"{models_path}{vector_size}/{model_num}.sav")
        
        # save prediction
        os.makedirs(f"{pred_path}{vector_size}", exist_ok=True)
        with open(f"{pred_path}{vector_size}/{model_num}.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(pred)

100%|██████████| 14/14 [1:31:11<00:00, 390.82s/it] 
